In [1]:
from chatTool import *
from Net import *
import pickle
import random
import os

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from torch import optim

use_cuda = torch.cuda.is_available()

LangBag = "dict.pkl"
DataName = "./dgk_lost_conv/results/lost.conv.tconv"
ModelDir = "lost"
# EncoderFile = "EncoderEos.pth"
# DecoderFile = "DecoderEos.pth"
# EncoderFile = "LostEncoder.pth"
# DecoderFile = "LostDecoder.pth"
EncoderFile = os.path.join(ModelDir, "Encoder.pth")
DecoderFile = os.path.join(ModelDir, "Decoder.pth")

with open(LangBag, 'rb') as f:
    lang = pickle.load(f)

trainset = Reader(DataName)

Load file:./dgk_lost_conv/results/lost.conv.tconv, Size: 47496



In [2]:
INPUTSIZE = len(lang.index2word)

if os.path.isfile(EncoderFile):
    encoder = torch.load(EncoderFile)
    print("Load encoder model: {}.".format(EncoderFile))
else:
    encoder = EncoderRNN(INPUTSIZE, 64, 256)
    print("Create encoder model.")

if os.path.isfile(DecoderFile):
    decoder = torch.load(DecoderFile)
    print("Load decoder model: {}.".format(DecoderFile))
else:
    decoder = DecoderRNN(INPUTSIZE, 64, 256, INPUTSIZE)
    print("Create decoder model.")

if use_cuda:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

Load encoder model: lost/Encoder.pth.
Load decoder model: lost/Decoder.pth.


In [3]:
def predit(q):
    input = torch.Tensor(lang.sentenceToVector(q)).long()
    input = Variable(input)

    if use_cuda:
        input = input.cuda()
    o, encoder_hidden = encoder(input)
    decoder_input = Variable(torch.LongTensor([0]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    decoder_hidden = o.view(1,1,-1)
    ans = []
    for i in range(50):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0]
        ans.append(ni)
        decoder_input = Variable(torch.LongTensor([ni]))
        if use_cuda:
            decoder_input = decoder_input.cuda()   
        if ni == 1:
            break
    return lang.vectorToSentence(ans)

In [6]:
for _ in range(10):
    q,a = random.choice(trainset)
    print("Q: {}\nA: {}\nM: {}\n".format(q, a, predit(q)))

Q: 回你的教堂去
A: 別上飛機  Eko
M: 你想讓我死嗎?

Q: 我什麼都不知道
A: 你知道她的那張疑犯照片
M: 他們是從哪兒來的?

Q: 你們不介意吧?
A: 好的
M: 我們在澳大利亞

Q: 你上完了 快開門
A: Austen 我發誓
M: 我們把一個牧師鎖在了外面?

Q: 太好看了
A: 謝謝
M: 我們可以靠他們

Q: 我只是...
A: 我只是很害怕
M: 我們來這兒之前

Q: 這個女人會告訴你如何回來
A: 誰...她是誰?
M: 你想知道他們怎麼會那樣?

Q: 什麼?
A: 你想打開它?
M: 我們在澳大利亞

Q: 我又和另一女子發生了關係
A: 完事以後  我又看她們倆...
M: 我們去找Locke

Q: 我們非常安全  好吧
A: 所以在這裏別見外
M: 我們已經獲救了



In [5]:
print(predit("我們走"))

你真以爲他會告訴我們?


In [6]:
print(predit("打球"))

該你了


In [14]:
input = torch.Tensor(lang.sentenceToVector("我們非常安")).long()
input = Variable(input)

if use_cuda:
    input = input.cuda()

In [15]:
output = encoder.embedding(input.long()).view(1, -1, encoder.em_size)
output, hidden = encoder.gru(output)

In [13]:
output[:,-2,:]

Variable containing:
 0.2507  0.6256  0.0267  ...  -0.0063 -0.0093  0.1319
[torch.cuda.FloatTensor of size 1x1024 (GPU 0)]

In [16]:
output[:,-1,:]

Variable containing:
 0.2507  0.6256  0.0267  ...  -0.0063 -0.0093  0.1319
[torch.cuda.FloatTensor of size 1x1024 (GPU 0)]